In [2]:
import googlemaps
import pandas as pd
import os
import json
import requests
from pprint import pprint


In [4]:
def load_api_keys(file_path='api_keys.json'):
    """
    Load API keys from a JSON file.
    """
    with open(file_path, 'r') as file:
        keys = json.load(file)
    return keys

In [10]:
keys = load_api_keys('api_keys.json')

In [8]:
def initialize_google_maps_client(api_key):
    """
    Initialize the Google Maps client.
    """
    return googlemaps.Client(key=api_key)

In [14]:
gm_client = initialize_google_maps_client(keys['google_maps_api_key'])


In [17]:
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping for wide DataFrames
pd.set_option('display.max_colwidth', None)  # Show full column content
location = '29.9511,-90.0715'


In [19]:

def get_nearby_restaurants_google(gm_client, location, radius=1000):
    """
    Fetch nearby restaurants using the Google Maps API.
    """
    nearby_places = gm_client.places_nearby(location=location, radius=radius, type='restaurant')
    df = pd.DataFrame([{
        'Name': place['name'].strip().lower(),
        'Google Business ID': place.get('place_id'),
        'Google Address': place.get('vicinity'),
        'Google Rating': place.get('rating'),
        'Google Total Ratings': place.get('user_ratings_total'),
        'Google Price': place.get('price_level'),
        # Add Photos column with an empty list as default
        'Photos': []
    } for place in nearby_places['results']])

    # Populate the Photos column with the list of photo references for each place
    for index, row in df.iterrows():
        place_id = row['Google Business ID']
        place_details = gm_client.place(place_id=place_id)
        photos = place_details.get('result', {}).get('photos', [])
        # Add list of photo references to the 'Photos' column
        df.at[index, 'Photos'] = [photo['photo_reference'] for photo in photos]

    return df

In [23]:
google_df = get_nearby_restaurants_google(gm_client, location)

In [24]:
print(google_df)

                                                 Name           Google Business ID                         Google Address  Google Rating  Google Total Ratings  Google Price                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [25]:
def get_nearby_restaurants_yelp(api_key, location, radius=1000):
    """
    Fetch nearby restaurants using the Yelp Fusion API.
    """
    # Yelp API endpoint for business search
    url = "https://api.yelp.com/v3/businesses/search"

    # Convert radius from meters to miles for Yelp API (as Yelp uses miles for distance)

    # Split the location into latitude and longitude
    lat, lon = location.split(',')

    # Set parameters for the Yelp API request
    headers = {
        'Authorization': f'Bearer {api_key}'
    }
    params = {
        'term': 'restaurant',
        'latitude': lat,
        'longitude': lon,
        'radius': radius,  # Convert back to meters for Yelp's radius parameter
        'limit': 50  # Yelp API allows up to 50 results per request
    }

    # Make a request to the Yelp API
    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Error: Unable to fetch data from Yelp API. Status Code: {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

    data = response.json()

    # Create DataFrame from Yelp API response
    df = pd.DataFrame([{
        'Name': business['name'],
        'Yelp Business ID': business.get('id'),
        'Yelp Address': ', '.join(business['location'].get('display_address', [])),
        'Yelp Rating': business.get('rating'),
        'Yelp Total Ratings': business.get('review_count'),
        'Yelp Price': business.get('price', None),  # Price may not be available for all businesses
        'Yelp Cuisine': business.get('categories')
    } for business in data.get('businesses', [])])
    
    return df

In [26]:
yelp_df = get_nearby_restaurants_yelp(keys['yelp_api_key'], location)


In [27]:
print(yelp_df)

                               Name        Yelp Business ID                                                                Yelp Address  Yelp Rating  Yelp Total Ratings Yelp Price                                                                                                                                                 Yelp Cuisine
0   Gallier Restaurant & Oyster Bar  swVdc9piXvG0AXwL-F0GfQ                                    129 Carondelet St, New Orleans, LA 70112          4.3                1604         $$                          [{'alias': 'seafood', 'title': 'Seafood'}, {'alias': 'cajun', 'title': 'Cajun/Creole'}, {'alias': 'southern', 'title': 'Southern'}]
1                              Luke  GBTPC53ZrG1ZBY3DT8Mbcw                                333 Saint Charles Ave, New Orleans, LA 70130          4.2                5205        $$$                                [{'alias': 'french', 'title': 'French'}, {'alias': 'german', 'title': 'German'}, {'alias': 'cajun', 'title': 'Cajun

In [28]:
def download_photos_from_dataframe(gm_client, df):
    """
    Download and save photos for each restaurant from the DataFrame.
    """
    for index, row in df.iterrows():
        place_name = row['Name']
        photo_references = row['Photos']  # Get the list of photo references from the DataFrame

        # Replace any illegal characters in folder names
        folder_name = ''.join(char for char in place_name if char.isalnum() or char in (' ', '_')).strip()

        # Create a directory named after the restaurant
        os.makedirs(f'photos/{folder_name}', exist_ok=True)

        for i, ref in enumerate(photo_references):
            # Retrieve the photo using the Google Maps client
            response = gm_client.places_photo(photo_reference=ref, max_width=1000)
            
            # Construct the file name with the source
            file_name = f"photos/{folder_name}/{folder_name}_Google_photo_{i+1}.jpg"
            
            # Save the photo to disk
            with open(file_name, 'wb') as file:
                for chunk in response:
                    file.write(chunk)

            print(f"Saved photo {file_name}")


In [29]:
google_df['Name'] = google_df['Name'].str.strip().str.lower() 
yelp_df['Name'] = yelp_df['Name'].str.strip().str.lower() 


In [31]:
merged_df = pd.merge(google_df, yelp_df, on='Name', how='outer', indicator=True)

# Remove rows with matching names
non_matching_df = merged_df[merged_df['_merge'] != 'both']

# Drop the indicator column
non_matching_df.drop(columns=['_merge'])

,Name,Google Business ID,Google Address,Google Rating,Google Total Ratings,Google Price,Photos,Yelp Business ID,Yelp Address,Yelp Rating,Yelp Total Ratings,Yelp Price,Yelp Cuisine
0,the windsor court,ChIJPwv3u3KmIIYRLmJLwBqo4o8,"300 Gravier Street, New Orleans",4.7,1438.0,NaN,"[AXCi2Q5TZnSpI20jZgg2Sc81LWNOah9vmrz0V3uN1gAOXdECuDmVE0Rj0TcW8gsGRl8OAZTlVA8X0H0wvkHvWvDUeAF-AN8Et9skJ1yS4IALLUR-ajtkzbmpN4Isa2_HoABaJJ4IaIdIQL7GXoT89wt18MUYgHlEFS2gURgXQzIyonV_SgsO, AXCi2Q6_g3FWg7xj6SJgMRm97nUGJ8-6GNOSok5vOY1BNLQIMlPhDkhyXFnvVUqaaW3N6ua_TM7sVi3n0qx0KP3snvA24xrT7glZ4ksBUrgoYhdnrKhQJU3m2anBz7wtH21iizcRc17lkXLwGPxqkcIIgkejLxgO-IBqkkoyakwb-sjs7Umq, AXCi2Q5NaNAhTpg472HMBOf840Sd1OhaJISNr5nr_nOFHlXlgWTClAzcmVzinBwAfcMH19jfAAb22zl14084nneRM6GD8EB3I3_nqd3TM1mW4TXqtn59xBgvzS14KsAhv_4Z1ZTqZBXMaaAHXtRSRKhKrT4DrFGClcXsnrVNGRuPK_3Gk7j5, AXCi2Q6so307AuP5sbfj0FElFY8bCjJGajSmhKhArBlcn0mLtCct7FTX0gtB1mz7Utsi5iIr1qMUCUBKsxXLYs5hA8YRLVJMYZkFEQBfpF9-biXlrp3IQZqf0zpkxaE5vKMqe3UsqhY1-jJWNiTj-NJIgKSqLJ6Xe-srIyHC6sayW6lPSOnT, AXCi2Q42Q1TkT2P7TQ7RwPzWZSGH0jbsuzYe11apzV1uqNzin2eb7bBGtNzYXR3V2Sq-_geLWA0iMxm9wNUoV0JcMmAeXwPYY8EO91qbpoIWtIOt5AfdVlqhhrs8P5Xcl6s_5Zy4nc51i-b1TMfDaBZVXrbw9EGABkMAiKj8qqC7cWMkKlWV, AXCi2Q7y7pviaZz4kobYtiyJ14zYYsTN_4kWfIACxBiQV-w2anLw_p-pW1ztwx1FiRQcijdqGM90MWyW0-pACV-crSYEhBdkz2eEuENYPcQXypjt-OTPk8M9GfsmMFI-nKuRzZMEQFsH0fpKesjMYGl4ea57XhCSSwQIsbJ_67G4q66lxTsb, AXCi2Q6T9iB17LPyDcpogEP4ct6VuCf0SYwCVgEvOVYEOaQ00DTqxbVB2QdmQ7VI3vLzyZYx4LwrG594vL9gwaQq9rAyqXVOrWhJUs5DRJsO82TCMTnSdeEZx7NQrQw3hDWVEs_rX-21-SaB9f7EoxigM82uCwyN6Kxr5ewG-6DDrjN5AxNm, AXCi2Q7Jsl60l8oElnF3eqncQdQT9k2bY9NavYbXHUp9l0yMJQ_z2sz2CFqqgp3qwV4sgBcJM3QhR4s1KFGzjAtcDTEPOWODWEzqRO5diM7OA9e-hU4ra7RTAGzZeEGp7w2A0K7VmdjTHVWmLPKdxuSpBWkZ06PNwiEjWdHYym5uM02blDFc, AXCi2Q4TSUYKmFtgwAY4xCkn5arH0-epJm50wXOhcCn6xCCmYvMT30Tham-_DX2oaUSVyfM3ncyWtK3JOQgM6z18KciBo1_Y_YSaaG7e89tQ1deDYracTMeRPJZwiR5Dd8wDdHOmeUIU94MS8VhKk6zd92vfqRSYh0cc8goxhZpLbVaGNS5R, AXCi2Q639xsbYzQ6n4dSe0Q6IDNCtPyL7DONa2HEx3pgFn7GwgDfZ0svmXxOJSApyXL6TtE_YhNY2R3xABHfmXmZuslUYjqwA3d4WPqFBzUU9JFrOiqYPYatf8hWai1okeIA99fp4Tmurl_tBi3aIWi1nrbCfN6lPHBMIbU0YgmuPUOTXwbX]",NaN,NaN,NaN,NaN,NaN,NaN
1,bourbon house,ChIJs_quaAymIIYRUwRs2-LHWqA,"144 Bourbon Street, New Orleans",4.3,5015.0,3.0,"[AXCi2Q5Rl0o91AtRYOOvc8d7yN2LjOY8Wf0lju4sCtafZgwhrOIn9AyK_7W1pAe4So3zrgsdHsTPKFzCE_x7ZZ1A6xMFUVomSG23avCHfj8FIfTQRapEpZRKPmzKIGiAzsA339GOpQMfsZZS4izF1MHrZGKyQNv5M0CMACLbxG0rmT32-_ON, AXCi2Q4vNCx3tO8GzA65oxVevAInVWyewceUxmdtPkzTvW-U9LlthJ9coRxQHInPMw9_C7FwPyqARJkQR89cd1Rwp2CJTzLY0GOfQeGmNjNUklFzqZxIen5SksOGkA7bLPD3qhJfZWi09sTjFLOukgyemC2Yqj0_yB9fUrXa88BsG601xO0K, AXCi2Q7Up0hcJ_RWinIDyFF30eDUTLCNRgsjWIMW_CgqTk8U0Km14q8ujFiYMe6gB-zO00KBxZwqr76mAlL8_xQFttpaRIYSyUQ8ESaHXzlX6vKvsMjjjTXAcoHnw1neJgaKStvXeMDgI_TZz3_yvl3fdWQK6nYx9ZirQ0mXCMl3nTujrMZN, AXCi2Q4JOFy5iJfNBUzcGwFAJtKHvCXkYuCLg4A0gKWw4lp3YB1m-zC9acDqxZHQeHPSm_U9Bv9Kk7uoppvssIskIOim2-8vauhr11Vr1RFC2JyltOCJf0qhME_yoVCWU-2bLMBDlA4EQcMZxhV_OvUTmEZqAWp2Dy-hNRut7c6MO_aCWtEz, AXCi2Q6LRqYa34BlHmPfxt9Erkea5j--QDgdd5Wy_7SMX4AHoX7wjZy86HVHR-Fq32UUqufALFFTaN8X5zUvladakRuEGSLPqRYaTaGT0fwTg3G_AtRLRAY2dsEfrIxitEWFZvmAWM5uvPaq25FEXcjAtyQlrRv00mHgdHD_qvfzsyWXXaLr, AXCi2Q5X3dRsFGJVDjhfKskuiR5_M12SglsvmjCCF-rWuT_vKiQd4K72HlOsu450l7CadgSuYln6B0WnUwy_7aSEVpMO1Flys3lovs-O-qcgmfyABzCrak1y7D65GS1XMOqd_K9rfANed_P1xhr8CepHmTcX9drsIlqx7-7aRLgYKeyueEF2, AXCi2Q7HGk--jQ6ujvGxXlI4B-rvxXCmNMzRq1hIay6vOlJZvd5pBqRbzuFzoTmG8_tZzm8wI9y2ttCxAbGDes78aCTyH6ls05hqKiByq-Via16Zn8K8e4gphAU2vq_IoKPUMPA8QM_oIf5pUgR-RU2GRUl-GPjW93JoZQ1MgogH5E7TlYmb, AXCi2Q6lgc6JkdvoZDchJ0Sx12PzTB6C6314TEEEci5V0eSSkFLDleQFw6-yJ_-tssg6KjwkdOcrS9FdpL-n5DWSYylb_rq9Htc2QmpcgwiiI6d4heG30rbJnEMwszVblxPGZhOahnTINJbI7CQLzw_DYWmxw9VoTebRfVM17bGkt_MDounB, AXCi2Q58yzYP-PKVMy62rhlPbssrOExOgiON60TjwBl4b72QjWRf47_6FG0ZhgwBelu2L_GVjHJ3VFCfgDP9ctSh00Y2cApyd41TEpWSnljIK9XukUtBrLiJAHqVjIQXq5YX_XY-jr2d8PxZhV3DtLH8rFD7hw_kdcn3DBYuj6_JdCaSdHCX, AXCi2Q5WW72bY_VEsOv6Ku21112wisa2PfMtlxSmg7yhS46jJeAYJY23NdnG0HrlV-fAVpukC8CqwjLk6Id0IAyTsA2jQH8Rlu3Ah6mpkMxbdSyD2Xkvo5xgbZzNKtPzmROdpqfboMACvID

In [32]:
non_matching_df.head()

,Name,Google Business ID,Google Address,Google Rating,Google Total Ratings,Google Price,Photos,Yelp Business ID,Yelp Address,Yelp Rating,Yelp Total Ratings,Yelp Price,Yelp Cuisine,_merge
0,the windsor court,ChIJPwv3u3KmIIYRLmJLwBqo4o8,"300 Gravier Street, New Orleans",4.7,1438.0,NaN,"[AXCi2Q5TZnSpI20jZgg2Sc81LWNOah9vmrz0V3uN1gAOXdECuDmVE0Rj0TcW8gsGRl8OAZTlVA8X0H0wvkHvWvDUeAF-AN8Et9skJ1yS4IALLUR-ajtkzbmpN4Isa2_HoABaJJ4IaIdIQL7GXoT89wt18MUYgHlEFS2gURgXQzIyonV_SgsO, AXCi2Q6_g3FWg7xj6SJgMRm97nUGJ8-6GNOSok5vOY1BNLQIMlPhDkhyXFnvVUqaaW3N6ua_TM7sVi3n0qx0KP3snvA24xrT7glZ4ksBUrgoYhdnrKhQJU3m2anBz7wtH21iizcRc17lkXLwGPxqkcIIgkejLxgO-IBqkkoyakwb-sjs7Umq, AXCi2Q5NaNAhTpg472HMBOf840Sd1OhaJISNr5nr_nOFHlXlgWTClAzcmVzinBwAfcMH19jfAAb22zl14084nneRM6GD8EB3I3_nqd3TM1mW4TXqtn59xBgvzS14KsAhv_4Z1ZTqZBXMaaAHXtRSRKhKrT4DrFGClcXsnrVNGRuPK_3Gk7j5, AXCi2Q6so307AuP5sbfj0FElFY8bCjJGajSmhKhArBlcn0mLtCct7FTX0gtB1mz7Utsi5iIr1qMUCUBKsxXLYs5hA8YRLVJMYZkFEQBfpF9-biXlrp3IQZqf0zpkxaE5vKMqe3UsqhY1-jJWNiTj-NJIgKSqLJ6Xe-srIyHC6sayW6lPSOnT, AXCi2Q42Q1TkT2P7TQ7RwPzWZSGH0jbsuzYe11apzV1uqNzin2eb7bBGtNzYXR3V2Sq-_geLWA0iMxm9wNUoV0JcMmAeXwPYY8EO91qbpoIWtIOt5AfdVlqhhrs8P5Xcl6s_5Zy4nc51i-b1TMfDaBZVXrbw9EGABkMAiKj8qqC7cWMkKlWV, AXCi2Q7y7pviaZz4kobYtiyJ14zYYsTN_4kWfIACxBiQV-w2anLw_p-pW1ztwx1FiRQcijdqGM90MWyW0-pACV-crSYEhBdkz2eEuENYPcQXypjt-OTPk8M9GfsmMFI-nKuRzZMEQFsH0fpKesjMYGl4ea57XhCSSwQIsbJ_67G4q66lxTsb, AXCi2Q6T9iB17LPyDcpogEP4ct6VuCf0SYwCVgEvOVYEOaQ00DTqxbVB2QdmQ7VI3vLzyZYx4LwrG594vL9gwaQq9rAyqXVOrWhJUs5DRJsO82TCMTnSdeEZx7NQrQw3hDWVEs_rX-21-SaB9f7EoxigM82uCwyN6Kxr5ewG-6DDrjN5AxNm, AXCi2Q7Jsl60l8oElnF3eqncQdQT9k2bY9NavYbXHUp9l0yMJQ_z2sz2CFqqgp3qwV4sgBcJM3QhR4s1KFGzjAtcDTEPOWODWEzqRO5diM7OA9e-hU4ra7RTAGzZeEGp7w2A0K7VmdjTHVWmLPKdxuSpBWkZ06PNwiEjWdHYym5uM02blDFc, AXCi2Q4TSUYKmFtgwAY4xCkn5arH0-epJm50wXOhcCn6xCCmYvMT30Tham-_DX2oaUSVyfM3ncyWtK3JOQgM6z18KciBo1_Y_YSaaG7e89tQ1deDYracTMeRPJZwiR5Dd8wDdHOmeUIU94MS8VhKk6zd92vfqRSYh0cc8goxhZpLbVaGNS5R, AXCi2Q639xsbYzQ6n4dSe0Q6IDNCtPyL7DONa2HEx3pgFn7GwgDfZ0svmXxOJSApyXL6TtE_YhNY2R3xABHfmXmZuslUYjqwA3d4WPqFBzUU9JFrOiqYPYatf8hWai1okeIA99fp4Tmurl_tBi3aIWi1nrbCfN6lPHBMIbU0YgmuPUOTXwbX]",NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,bourbon house,ChIJs_quaAymIIYRUwRs2-LHWqA,"144 Bourbon Street, New Orleans",4.3,5015.0,3.0,"[AXCi2Q5Rl0o91AtRYOOvc8d7yN2LjOY8Wf0lju4sCtafZgwhrOIn9AyK_7W1pAe4So3zrgsdHsTPKFzCE_x7ZZ1A6xMFUVomSG23avCHfj8FIfTQRapEpZRKPmzKIGiAzsA339GOpQMfsZZS4izF1MHrZGKyQNv5M0CMACLbxG0rmT32-_ON, AXCi2Q4vNCx3tO8GzA65oxVevAInVWyewceUxmdtPkzTvW-U9LlthJ9coRxQHInPMw9_C7FwPyqARJkQR89cd1Rwp2CJTzLY0GOfQeGmNjNUklFzqZxIen5SksOGkA7bLPD3qhJfZWi09sTjFLOukgyemC2Yqj0_yB9fUrXa88BsG601xO0K, AXCi2Q7Up0hcJ_RWinIDyFF30eDUTLCNRgsjWIMW_CgqTk8U0Km14q8ujFiYMe6gB-zO00KBxZwqr76mAlL8_xQFttpaRIYSyUQ8ESaHXzlX6vKvsMjjjTXAcoHnw1neJgaKStvXeMDgI_TZz3_yvl3fdWQK6nYx9ZirQ0mXCMl3nTujrMZN, AXCi2Q4JOFy5iJfNBUzcGwFAJtKHvCXkYuCLg4A0gKWw4lp3YB1m-zC9acDqxZHQeHPSm_U9Bv9Kk7uoppvssIskIOim2-8vauhr11Vr1RFC2JyltOCJf0qhME_yoVCWU-2bLMBDlA4EQcMZxhV_OvUTmEZqAWp2Dy-hNRut7c6MO_aCWtEz, AXCi2Q6LRqYa34BlHmPfxt9Erkea5j--QDgdd5Wy_7SMX4AHoX7wjZy86HVHR-Fq32UUqufALFFTaN8X5zUvladakRuEGSLPqRYaTaGT0fwTg3G_AtRLRAY2dsEfrIxitEWFZvmAWM5uvPaq25FEXcjAtyQlrRv00mHgdHD_qvfzsyWXXaLr, AXCi2Q5X3dRsFGJVDjhfKskuiR5_M12SglsvmjCCF-rWuT_vKiQd4K72HlOsu450l7CadgSuYln6B0WnUwy_7aSEVpMO1Flys3lovs-O-qcgmfyABzCrak1y7D65GS1XMOqd_K9rfANed_P1xhr8CepHmTcX9drsIlqx7-7aRLgYKeyueEF2, AXCi2Q7HGk--jQ6ujvGxXlI4B-rvxXCmNMzRq1hIay6vOlJZvd5pBqRbzuFzoTmG8_tZzm8wI9y2ttCxAbGDes78aCTyH6ls05hqKiByq-Via16Zn8K8e4gphAU2vq_IoKPUMPA8QM_oIf5pUgR-RU2GRUl-GPjW93JoZQ1MgogH5E7TlYmb, AXCi2Q6lgc6JkdvoZDchJ0Sx12PzTB6C6314TEEEci5V0eSSkFLDleQFw6-yJ_-tssg6KjwkdOcrS9FdpL-n5DWSYylb_rq9Htc2QmpcgwiiI6d4heG30rbJnEMwszVblxPGZhOahnTINJbI7CQLzw_DYWmxw9VoTebRfVM17bGkt_MDounB, AXCi2Q58yzYP-PKVMy62rhlPbssrOExOgiON60TjwBl4b72QjWRf47_6FG0ZhgwBelu2L_GVjHJ3VFCfgDP9ctSh00Y2cApyd41TEpWSnljIK9XukUtBrLiJAHqVjIQXq5YX_XY-jr2d8PxZhV3DtLH8rFD7hw_kdcn3DBYuj6_JdCaSdHCX, AXCi2Q5WW72bY_VEsOv6Ku21112wisa2PfMtlxSmg7yhS46jJeAYJY23NdnG0HrlV-fAVpukC8CqwjLk6Id0IAyTsA2jQH8Rlu3Ah6mpkMxbdSyD2Xkvo5xgbZzNKt

In [62]:
def get_yelp_food_and_drink_insights(api_key, business_id):
    """
    Fetch food and drink insights from Yelp using the Business ID.
    """
    url = f"https://api.yelp.com/v3/businesses/{business_id}/insights/food_and_drinks"
    print(url)
    headers = {
        'Authorization': f'Bearer {api_key}',
        'accept': 'application/json'
    }
    params = {
        'locale': 'en_US',
    }


    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: Unable to fetch data from Yelp for Business ID {business_id}. Status Code: {response.status_code}")
        return None


In [55]:
def fetch_and_print_yelp_insights(api_key, df):
    """
    Fetch Yelp food and drink insights for each business in the DataFrame and print them.
    """
    for index, row in df.iterrows():
        yelp_business_id = row.get('Yelp Business ID')  # Ensure the column name matches
        
        # Check if Yelp Business ID is not NaN
        if pd.notna(yelp_business_id):
            print(yelp_business_id)

            # Fetch Yelp insights
            insights = get_yelp_food_and_drink_insights(api_key, yelp_business_id)

            # Print insights if available
            if insights:
                print(f"Food and Drink Insights for {row['Name']} (Yelp Business ID: {yelp_business_id}):")
                print(json.dumps(insights, indent=4))  # Pretty-print the JSON data
            else:
                print(f"No insights available for {row['Name']}.")
        else:
            print(f"No Yelp Business ID for {row['Name']}.")


In [63]:
fetch_and_print_yelp_insights(keys['yelp_api_key'], non_matching_df)
    

No Yelp Business ID for the windsor court.
No Yelp Business ID for bourbon house.
No Yelp Business ID for mother's restaurant.
No Yelp Business ID for dickie brennan's steakhouse.
No Yelp Business ID for deanie's seafood restaurant in the french quarter.
No Yelp Business ID for galatoire's.
No Yelp Business ID for august.
No Yelp Business ID for emeril's.
No Yelp Business ID for antoine's restaurant.
No Yelp Business ID for mr. b's bistro.
No Yelp Business ID for ruth's chris steak house.
No Yelp Business ID for palace café by dickie brennan & co..
No Yelp Business ID for the court of two sisters.
No Yelp Business ID for the grill room at windsor court.
No Yelp Business ID for restaurant r'evolution.
swVdc9piXvG0AXwL-F0GfQ
https://api.yelp.com/v3/businesses/swVdc9piXvG0AXwL-F0GfQ/insights/food_and_drinks
Error: Unable to fetch data from Yelp for Business ID swVdc9piXvG0AXwL-F0GfQ. Status Code: 403
No insights available for gallier restaurant & oyster bar.
-FKFtavlZ7897Slw5A6ARQ
https:/